In [2]:
from Keys import my_google_api_key
import os
os.environ["GOOGLE_API_KEY"] = my_google_api_key

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
import langchain

In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [5]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.vectorstores import FAISS

In [6]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [7]:
vector_db=FAISS.load_local("faiss_vector_db",embeddings,allow_dangerous_deserialization=True)

In [13]:
my_query="Describe his academic journey?"

In [9]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

In [3]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that answers queries about a candidate's academic and internship experience. Answer the questions while framing the candidate's experience and skills in the most positive way but do not make up any information beyond what is provided."
)

In [13]:
user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with drafting an answer for a question for a job application

The answer should only contain information on the candidate that has been available to you, do not falsify any skills or experience. 
For behavourial questions, use slightly creative vocabulary to best answer them.
The question is: {question}""",
    input_variables=["question"]
)

In [14]:
vector_db.similarity_search(query=my_query,k=5)

[Document(metadata={'source': 'C://Users//ASUS//Documents//Python Scripts//LnE Langchain//RAG Documents//CL Q.docx'}, page_content='I am highly motivated to apply for this role, leveraging my robust academic background in Data Science and Analytics, with a master’s from the University of Leeds and a BTech in Mathematics and Computing from Delhi Technological University. My hands-on experience in data analysis, predictive modelling, and statistical analysis, as demonstrated through my successful academic projects and internship at PwC, aligns perfectly with your job requirements. I have built my proficiency in Python, R, SQL, and MATLAB, by extensively working on building statistical models and performing in-depth mathematical analysis as required by the project specifications. I have always been passionate about research and translating mathematical theory into practice, ensuring high attention to detail and analytical problem-solving. I have continuously strived to analyse the minutes

In [15]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vector_db.as_retriever())

In [16]:
answer=chain({"question": my_query}, return_only_outputs=True)

In [17]:
answer

{'answer': 'He has a master’s degree in data science and analytics from the University of Leeds and a bachelor’s degree in mathematics and computing from Delhi Technological University.\n',
 'sources': 'C://Users//ASUS//Documents//Python Scripts//LnE Langchain//RAG Documents//CL Q.docx, C://Users//ASUS//Documents//Python Scripts//LnE Langchain//RAG Documents//CL.docx'}

## Prompting

In [15]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

In [9]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that answers queries about a candidate and helps with drafting answers for job applications."
)

In [13]:
user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with drafting an answer for a question for a job application

The answer should only contain information on the candidate that has been available to you, do not falsify any skills or experience. 
For behavourial questions, use slightly creative vocabulary to best answer them.
The question is: {question}""",
    input_variables=["question"]
)

In [12]:
question="Why do you want to join Airbus?"

In [16]:
prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

In [17]:
rag_chain = (
    {"context": vector_db.as_retriever(), "question": lambda x: x["question"]}
    | prompt
    | llm
)

In [18]:
answer=rag_chain.invoke({"question":question})

GoogleGenerativeAIError: Error embedding content: 'ProtoType' object has no attribute 'DESCRIPTOR'